In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [3]:
!pip uninstall -y torch torchvision torchaudio
!pip install --upgrade torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.6/955.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install pandas datasets

In [5]:
!pip install transformers peft datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.9 MB/s eta 0:00:00


In [8]:
import os
import json
import zipfile
import pandas as pd
import numpy as np
import fitz
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer,
    pipeline, DataCollatorWithPadding
)
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
import kagglehub

# **Model and Dataset Preprocessing**

In [9]:
# importing Model

checkpoint = "bigscience/bloomz-3b"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", torch_dtype="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

**Coverting into multi turn chat dataset**



In [10]:
# Dataset preparation
multi_turn_data = []
keywords = ["diabetes", "blood sugar", "glucose", "insulin", "a1c", "hypoglycemia", "hyperglycemia", "metformin", "type 1", "type 2"]

# 1. MedQuAD
medquad_df = pd.read_csv("/content/medDataset_processed.csv")
medquad_mask = medquad_df['Question'].fillna('').str.lower().apply(lambda x: any(k in x for k in keywords)) | \
               medquad_df['Answer'].fillna('').str.lower().apply(lambda x: any(k in x for k in keywords))
filtered_medquad = medquad_df[medquad_mask]
for i in range(0, len(filtered_medquad), 4):
    session = filtered_medquad.iloc[i:i+4]
    if len(session) < 2: continue
    dialog = []
    for _, row in session.iterrows():
        dialog.append({"role": "user", "content": row["Question"]})
        dialog.append({"role": "assistant", "content": row["Answer"]})
    multi_turn_data.append({"messages": dialog})

# 2. MedicationQA
medqa_dataset = load_dataset("truehealth/medicationqa")
medqa_diabetes = medqa_dataset['train'].filter(lambda x: any(k in x['Question'].lower() for k in keywords))
for item in medqa_diabetes:
    multi_turn_data.append({"messages": [
        {"role": "user", "content": item['Question']},
        {"role": "assistant", "content": item['Answer']}
    ]})

# 3. MedQA
us_medqa_path = "/content"
us_files = ["train.jsonl", "dev.jsonl", "test.jsonl", "US_qbank.jsonl", "US_exam.jsonl", "US_test_sampled.jsonl", "US_valid_sampled.jsonl"]
for file in us_files:
    path = os.path.join(us_medqa_path, file)
    if os.path.exists(path):
        with open(path, "r") as f:
            for line in f:
                data = json.loads(line)
                if any(k in data.get("question", "").lower() or k in data.get("answer", "").lower() for k in keywords):
                    multi_turn_data.append({"messages": [
                        {"role": "user", "content": data["question"]},
                        {"role": "assistant", "content": data["answer"]}
                    ]})

# 4. MedDialog-EN
path = kagglehub.dataset_download("xuehaihe/medical-dialogue-dataset")
dialog_path = os.path.join(path, "medical_dialogue_en.json")
if os.path.exists(dialog_path):
    with open(dialog_path, "r") as f:
        dialog_data = json.load(f)
    for conv in dialog_data:
        if "dialogue" in conv and any(k in conv.get("disease", "").lower() for k in keywords):
            messages = []
            for turn in conv["dialogue"].split("##"):
                if ":" in turn:
                    role, content = turn.split(":", 1)
                    role = "user" if "patient" in role.lower() else "assistant"
                    messages.append({"role": role, "content": content.strip()})
            if messages:
                multi_turn_data.append({"messages": messages})

# 5. Synthetic QA
synthetic_examples = [
    {"messages": [
        {"role": "user", "content": "Can I take insulin and metformin together?"},
        {"role": "assistant", "content": "Yes, they are often prescribed together. Follow your doctor’s advice."}
    ]}
]
multi_turn_data.extend(synthetic_examples)

# Save dataset
with open("/content/drive/MyDrive/diabetes_multiturn_finetune_data.jsonl", "w") as f:
    for item in multi_turn_data:
        f.write(json.dumps(item) + "\n")
print(f"Total multi-turn examples: {len(multi_turn_data)}")

README.md:   0%|          | 0.00/541 [00:00<?, ?B/s]

(…)-00000-of-00001-7427a10e891759be.parquet:   0%|          | 0.00/215k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/690 [00:00<?, ? examples/s]

Filter:   0%|          | 0/690 [00:00<?, ? examples/s]

Total multi-turn examples: 4873


# **Finetuning BloomZ using PEFT(LoRA)**

In [12]:
# Tokenization
dataset = load_dataset("json", data_files="diabetes_multiturn_finetune_data.jsonl", split="train")
tokenizer.chat_template = "{% for message in messages %}" + \
                         "{% if message.role == 'user' %}{{bos_token}}Human: {{message.content}}{% endif %}" + \
                         "{% if message.role == 'assistant' %}{{eos_token}}Assistant: {{message.content}}{% endif %}" + \
                         "{% endfor %}"
def tokenize(example):
    prompt = tokenizer.apply_chat_template(example["messages"], tokenize=False)
    encoding = tokenizer(prompt, truncation=True, max_length=1024, return_tensors="pt", padding="max_length")
    input_ids = encoding["input_ids"][0]
    attention_mask = encoding["attention_mask"][0]
    labels = input_ids.clone()
    labels = torch.roll(labels, shifts=-1, dims=0)
    labels[-1] = -100
    return {"input_ids": input_ids, "labels": labels, "attention_mask": attention_mask}

tokenized_dataset = dataset.map(tokenize, remove_columns=["messages"])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4873 [00:00<?, ? examples/s]

In [14]:
# LoRA
lora_config = LoraConfig(r=8, lora_alpha=16, target_modules=["query_key_value"], lora_dropout=0.1, bias="none", task_type=TaskType.CAUSAL_LM)
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./finetuned_diabetes_bot", per_device_train_batch_size=2, gradient_accumulation_steps=4,
    learning_rate=2e-5, num_train_epochs=1, save_strategy="epoch", save_total_limit=1,
    logging_steps=20, fp16=True, gradient_checkpointing=True, report_to="none"
)

model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset, data_collator=DataCollatorWithPadding(tokenizer, padding="longest", max_length=1024))
trainer.train()

# Save model
trainer.save_model("/content/drive/MyDrive/finetuned_diabetes_bloomz_lora")
tokenizer.save_pretrained("/content/drive/MyDrive/finetuned_diabetes_bloomz_lora")


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,16.837900
40,16.052800
60,15.440600
80,14.903100
100,13.678400
120,12.069300
140,10.014400
160,8.024400
180,6.121300
200,4.594200


('/content/drive/MyDrive/finetuned_diabetes_bloomz_lora/tokenizer_config.json',
 '/content/drive/MyDrive/finetuned_diabetes_bloomz_lora/special_tokens_map.json',
 '/content/drive/MyDrive/finetuned_diabetes_bloomz_lora/tokenizer.json')